## 네이버 검색 결과 크롤링
- 영등포역 맛집
- Selenium 이용

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote

- request.get()으로 데이터를 가져오는 경우

In [2]:
base_url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum'
url = f'{base_url}&query={quote("영등포역 맛집")}'
res = requests.get(url)
soup = BeautifulSoup(res.text)

In [3]:
lis = soup.select('.bx._svp_item')
len(lis)        # 페이지다운 하지 않은 현재 보이는 데이터 갯수

30

- Selenium 이용해서 데이터를 가져오는 경우

In [4]:
from selenium import webdriver

driver = webdriver.Chrome()     # PATH 설정 할 것 (chromedriver.exe가 위치해있는곳을 path 설정해야함)
driver.get(url)

In [5]:
# View 페이지를 아래로 내려서 많은 데이터를 확보
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

body = driver.find_element(By.TAG_NAME, 'body')     #body 태그를 찾는다
for _ in range(30):   
    body.send_keys(Keys.PAGE_DOWN)  #페이지 다운키를 보낸다
    time.sleep(1)       #1초씩 쉬었다 가기

In [6]:
soup = BeautifulSoup(driver.page_source)
lis = soup.select('.bx._svp_item')
len(lis)        # 페이지 다운을 해서 나온 데이터 갯수

110

In [7]:
driver.close()

- 제목, 내용 크롤링

In [21]:
title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
title

'영등포역 맛집 영등갈비 미묘하게 매력적인 고기집'

In [10]:
content = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
content[:200]

'최근에 영등포역 맛집 영등갈비 방문 후 포스팅 해봅니다 난생 처음 먹어본 색다른 갈비 였네요 영등포역... 서울 영등포구 영신로 19번길 1 영업시간: 오후 12시 - 밤 10시 브레이크타임: 오후 2시 - 오후 4시 (평일) 연락처: 0507...'

- 날짜 크롤링

In [8]:
li = lis[0]
date = li.select_one('.sub_time.sub_txt').get_text().strip()       #공백도 . 찍어줘야함
date

'2023.08.28.'

In [11]:
from datetime import datetime, timedelta

now = datetime.now()
now

datetime.datetime(2023, 9, 18, 15, 18, 15, 990975)

In [12]:
# 어제
date = now - timedelta(1)
f'{date.year}.{date.month:02d}.{date.day:02d}'

'2023.09.17'

In [13]:
# 2일 전
date = now - timedelta(2)
date.strftime('%Y.%m.%d')

'2023.09.16'

In [14]:
# 23시간 전
date = now - timedelta(hours=23)
date.strftime('%Y.%m.%d')

'2023.09.17'

In [15]:
# 40분 전
date = now - timedelta(minutes=40)
date.strftime('%Y.%m.%d')

'2023.09.18'

In [16]:
def convert_date(s):    # s는 40분 전 또는 '2023.08.28.' 같은 문자열
    if len(s) == 11:        #'2023.09.18.'
        return s[:-1]
    
    now = datetime.now()
    if '어제' in s:
        date = now - timedelta(1)
    elif '일' in s:         # 2일 전~ 7일 전
        date = now - timedelta(int(s[0]))
    elif '시간' in s:       #1 시간 전 ~ 23시간 전
        date = now - timedelta(hours=int(s[:-4]))       # -4: 뒤에서 부터 "시간 전" 없이 앞에 숫자만 나타나도록!
    elif '분' in s:         # 1분 전 ~ 59분 전
        date = now - timedelta(minutes=int(s[:-3]))
        
    return date.strftime('%Y.%m.%d')

- Step 4

In [23]:
data = []
for li in lis:
    date = convert_date(li.select_one('.sub_time.sub_txt').get_text().strip())
    title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
    content = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
    data.append({'날짜':date, '제목':title, '내용':content})


- Step 7

In [24]:
df = pd.DataFrame(data)
df

,날짜,제목,내용
0,2023.08.28,영등포역 맛집 영등갈비 미묘하게 매력적인 고기집,최근에 영등포역 맛집 영등갈비 방문 후 포스팅 해봅니다 난생 처음 먹어본 색다른 갈...
1,2023.09.08,영등포역 맛집 남다른 식감의 특수부위 영등포꼬들집,안녕하세요~ 두덩입니다 :) 오늘은 영등포역 맛집을 소개하려고 해요. 영등포꼬들집이...
2,2023.08.31,영등포역 맛집 추천 - 고기집 항정사이 영등포점 가성비 좋아요,#영등포역맛집 #영등포역고기집 #항정사이 영등포역 맛집 거리 지날 때마다 늘 손님들...
3,2023.05.28,청기와타운 - 영등포역 맛집 영등포 수원왕갈비 한정식 고깃집 추천,영업시간: 11:30 - 22:00 라스트 오더: 21:00 연락처: 0507-13...
4,2023.09.17,"[영등포역 맛집/송죽장] 70년 전통 노포 화상 중국집 고추짬뽕,탕수육,군만두(ft...",서울에서 손꼽히는 노포 화상 중국집중 한곳인 #영등포맛집 #영등포역맛집 #송죽장 7...
...,...,...,...
105,2023.09.10,[음식점리뷰]영등포역 맛집 교촌치킨에서 맛있는 치킨 먹어볼까요?,그래서 퇴근하자마자 집 근처에 있는 영등포역 맛집 교촌치킨으로 직행했다. 저번에 테...
106,2023.09.06,영등포역 인근 맛집 추천해주세요!!!!,영등포역 인근 맛집 추천해주세요!!!!
107,2023.09.18,영등포역맛집 이고집쭈꾸미 여전히맛있어 최고야,이고집쭈꾸미 본점 #영등포역맛집 #영등포맛집 #이고집쭈꾸미 #영등포쭈꾸미 친구들이랑...
108,2023.04.03,꽃구경 코스로 좋은 영등포역 맛집 광부맥주,꽃구경 코스로 좋은 영등포역 맛집 광부맥주 주말에 집에있다가 날씨가 너무 좋아서 당...
